# Behavioral Patterns

See also:
- https://github.com/faif/python-patterns
- https://sourcemaking.com/design_patterns/behavioral_patterns

## Observer

Define a one-to-many dependency between objects so that when one object
changes state, all its dependents are notified and updatedautomatically.

In [5]:
class Subject(object): #Represents what is being 'observed'
    def __init__(self):
        self._observers = [] # This where references to all the observers are being kept
                             # Note that this is a one-to-many relationship: there will be one subject to be observed by multiple _observers

    def attach(self, observer):
        if observer not in self._observers: #If the observer is not already in the observers list
            self._observers.append(observer) # append the observer to the list

    def detach(self, observer): #Simply remove the observer
        try:
            self._observers.remove(observer)
        except ValueError:
            pass

    def notify(self):
        for observer in self._observers:
            observer.update(self) # Alert the observers!

class Core(Subject): #Inherits from the Subject class

    def __init__(self, name=""):
        Subject.__init__(self)
        self._name = name #Set the name of the core
        self._temp = 0 #Initialize the temperature of the core

    @property #Getter that gets the core temperature
    def temp(self):
        return self._temp

    @temp.setter #Setter that sets the core temperature
    def temp(self, temp):
        self._temp = temp
        self.notify() #Notify the observers whenever somebody changes the core temperature

class TempViewer(object):
    def update(self, subject): #Alert method that is invoked when the notify() method in a concrete subject is invoked
        print("Temperature Viewer: {} has Temperature {}".format(subject._name, subject._temp))

In [7]:
#Let's create our subject
c1 = Core("Core 1")

#Let's create our observers
v1 = TempViewer()
v2 = TempViewer()

#Let's attach our observers to the first core
c1.attach(v1)
c1.attach(v2)

#Let's change the temperature of our first core
c1.temp = 80
c1.temp = 90

Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 90
Temperature Viewer: Core 1 has Temperature 90


## Visitor

Represent an operation to be performed on the elements of an object
structure. Visitor lets you define a new operation without changing the
classes of the elements on which it operates.

Note: Can be used to traverse the components of a structure build via Composite Pattern.

In [11]:
class House(object): #The class being visited 
    def accept(self, visitor):
        """Interface to accept a visitor"""
        visitor.visit(self) #Triggers the visiting operation!
        
        # For a composite structure you can make sure children would accept the visitor
        # for child in self.children():
             # child.accept(visitor)

    def work_on_hvac(self, hvac_specialist):
        print(self, "worked on by", hvac_specialist) #Note that we now have a reference to the HVAC specialist object in the house object!

    def work_on_electricity(self, electrician):
        print(self, "worked on by", electrician) #Note that we now have a reference to the electrician object in the house object!

    def __str__(self):
        """Simply return the class name when the House object is printed"""
        return self.__class__.__name__


class Visitor(object):
    """Abstract visitor"""
    def __str__(self):
        """Simply return the class name when the Visitor object is printed"""
        return self.__class__.__name__


class HvacSpecialist(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: HVAC specialist"""
    def visit(self, house):
        house.work_on_hvac(self) #Note that the visitor now has a reference to the house object


class Electrician(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: electrician"""
    def visit(self, house):
        house.work_on_electricity(self) #Note that the visitor now has a reference to the house object

In [10]:
#Create an HVAC specialist
hv = HvacSpecialist()
#Create an electrician
e = Electrician()

#Create a house
home = House()

#Let the house accept the HVAC specialist and work on the house by invoking the visit() method
home.accept(hv)

#Let the house accept the electrician and work on the house by invoking the visit() method
home.accept(e)

House worked on by HvacSpecialist
House worked on by Electrician


## Iterator

Provide a way to access the elements of an aggregate objects equentially
without exposing its underlying representation.

https://docs.python.org/3/library/itertools.html

In [33]:
from itertools import count, cycle, repeat, groupby, takewhile

for c in cycle('ABCD'):
    if c == 'D':
        break
    print(c)
    
for i in count(1):
    if i == 4:
        break
    print(i)
    
print(list(repeat(10, 3)))

keyfunc = lambda x: x[0]
for k, v in groupby(sorted([(1,2), (3,3), (4,5), (3,6), (1,9)], key=keyfunc), key=keyfunc):
    print(k, list(v))
    
print(list(takewhile(lambda x: x<5, [1,4,6,4,1])))

A
B
C
1
2
3
[10, 10, 10]
1 [(1, 2), (1, 9)]
3 [(3, 3), (3, 6)]
4 [(4, 5)]
[1, 4]


## Strategy
Define a family of algorithms, encapsulate each one, and make them
interchangeable. Strategy lets the algorithm vary independently from
clients that use it.

In [37]:
import abc

class Context:
    """
    Define the interface of interest to clients.
    Maintain a reference to a Strategy object.
    """

    def __init__(self, strategy):
        self._strategy = strategy

    def context_interface(self):
        self._strategy.algorithm_interface()


class Strategy(metaclass=abc.ABCMeta):
    """
    Declare an interface common to all supported algorithms. Context
    uses this interface to call the algorithm defined by a
    ConcreteStrategy.
    """

    @abc.abstractmethod
    def algorithm_interface(self):
        pass


class ConcreteStrategyA(Strategy):
    """
    Implement the algorithm using the Strategy interface.
    """

    def algorithm_interface(self):
        pass


class ConcreteStrategyB(Strategy):
    """
    Implement the algorithm using the Strategy interface.
    """

    def algorithm_interface(self):
        pass


concrete_strategy_a = ConcreteStrategyA()
context = Context(concrete_strategy_a)
context.context_interface()

## Chain of Responsibility
Avoid coupling the sender of a request to its receiver by giving
more than one object a chance to handle the request. Chain the receiving
objects and pass the request along the chain until an object handles it.

In [41]:
class Handler(object): #Abstract handler
    """Abstract Handler"""
    def __init__(self, successor):
        self._successor = successor # Define who is the next handler

    def handle(self, request):
            handled = self._handle(request) #If handled, stop here

            #Otherwise, keep going
            if not handled:
                self._successor.handle(request)    

    def _handle(self, request):
        raise NotImplementedError('Must provide implementation in subclass!')

class ConcreteHandler1(Handler): # Inherits from the abstract handler
    """Concrete handler 1"""
    def _handle(self, request):
        if 0 < request <= 10: # Provide a condition for handling
            print("Request {} handled in handler 1".format(request))
            return True # Indicates that the request has been handled

class DefaultHandler(Handler): # Inherits from the abstract handler
    """Default handler"""
    def _handle(self, request):
        """If there is no handler available"""
        #No condition checking since this is a default handler
        print("End of chain, no handler for {}".format(request))
        return True # Indicates that the request has been handled


handler = ConcreteHandler1(DefaultHandler(None))
for request in [2, 5, 30]:
    handler.handle(request)


Request 2 handled in handler 1
Request 5 handled in handler 1
End of chain, no handler for 30
